In [24]:
import pandas as pd
import numpy as np
import json
import glob
from urllib.request import urlopen
import wikipediaapi
import random 
from collections import Counter 
import re
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import metrics

In [25]:
# Pre-processing functions
def check_list(list_column, check_str):
    if check_str in list_column: 
        return True
    else:
        return False
    
def clean_text(text):
    text = text.replace('\nBULLET::::', ' ')
    text = text.replace('BULLET::::-', ' ')
    text = text.replace('BULLET::::', ' ')
    text = text.replace('\n', ' ')
    text = text.replace('\n\n', ' ')
    text = text.replace(r',', '')
    text = text.replace('.', '')
    text = text.replace(' - ', '')
    text = text.replace('-', '')
    text = text.replace('&nbsp;', ' ')
    text = text.replace('Page', ' ')
    text = text.replace(':', ' ')
    text = text.replace(';', ' ')
    text = text.replace('"', '')
    text = text.replace("'", '')
    text = text.replace('(', '')
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace(')', '')
    text = text.strip()
    text = re.sub(r'\d+.', ' ', text)
    text = re.sub(r' +', ' ', text)
    return text  

def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def str_replace(text, title):
    return text.replace(title + ' ' + title, '' + title + ' ', 1)  

def get_tamil_stop_words():
    swdf1 = pd.read_csv("data/TamilNLP_TamilStopWords.txt",  header=None) 
    swdf2 = pd.read_csv("data/custom_tamil_stop_words.txt",  header=None) 
    sw1 = swdf1[0].tolist()
    sw2 = swdf2[0].tolist()
    tamil_stop_words = list(set(sw1 + sw2))
    return tamil_stop_words

In [26]:
# Load the labeled data
data_files = glob.glob('data/tamil_wiki_data_with_parent_category/ready_data*')
df_list = []
for filename in data_files:
    df_list.append(pd.read_csv(filename))
    
df = pd.concat(df_list, ignore_index=True)    
df.shape

(110000, 8)

In [27]:
# drop duplicates in case any
df = df.drop_duplicates(subset='title', keep="first")
df.shape

(110000, 8)

In [28]:
# drop no category
is_NoCat =  df['parent_category'] != "NoCat"
df1 = df[is_NoCat]
df1.shape

(93517, 8)

In [29]:
# drop rows if it contains category crickter, because it is over represented in the data space
cricketer_flag = df1.apply(lambda row: check_list(row['categories'], "துடுப்பாட்டக்காரர்கள்"),axis=1)
df1 = df1.assign(is_cricketer=cricketer_flag.values) 
not_cricketer =  df1['is_cricketer'] == False
df1 = df1[not_cricketer]
df1.shape

(87753, 9)

In [30]:
# drop rows if it contains category temple, because it is over represented in the data space
temple_flag = df1.apply(lambda row: check_list(row['categories'], "கோயில்கள்"),axis=1)
df1 = df1.assign(is_temple=temple_flag.values) 
not_temple =  df1['is_temple'] == False
df1 = df1[not_temple]
df1.shape

(76305, 10)

In [31]:
# drop rows if there was difficulty in determining the parent category
is_depth_high =  df1['parent_category_recursion_depth'] < 12
df2 = df1[is_depth_high]
df2.shape

(70515, 10)

In [32]:
# Filter for articles with at least 500 category members
df3 = df2[df2['parent_category'].map(df2['parent_category'].value_counts()) > 500]
df3.shape

(68392, 10)

In [33]:
# Show the categories
df3.parent_category.value_counts()

புவியியல்      14692
வரலாறு          8725
சமூகம்          7296
அரசியல்         3925
உயிரியல்        3231
சமயம்           2844
பண்பாடு         2749
இலக்கியம்       2645
ஊடகவியல்        2508
தொழினுட்பம்     2237
வேதியியல்       2219
திரைப்படம்      2095
கணினியியல்      1913
கணிதம்          1663
மானிடவியல்      1400
இசை             1287
இயற்பியல்       1198
மொழி            1121
வானியல்          867
உளவியல்          706
கல்வி            686
வணிகவியல்        647
சட்டம்           638
நலம்             578
கட்டிடக்கலை      522
Name: parent_category, dtype: int64

In [34]:
# Equvalize the sample, over sample by small fraction
grouped = df3.groupby('parent_category')
df4 = grouped.apply(lambda x: x.sample(n=650, replace=True))
df4.parent_category.value_counts()

இசை            650
அரசியல்        650
கணினியியல்     650
சமயம்          650
திரைப்படம்     650
ஊடகவியல்       650
கட்டிடக்கலை    650
உளவியல்        650
சட்டம்         650
மானிடவியல்     650
சமூகம்         650
இலக்கியம்      650
புவியியல்      650
வேதியியல்      650
வரலாறு         650
நலம்           650
தொழினுட்பம்    650
பண்பாடு        650
உயிரியல்       650
வானியல்        650
இயற்பியல்      650
கணிதம்         650
மொழி           650
வணிகவியல்      650
கல்வி          650
Name: parent_category, dtype: int64

In [35]:
# Clean the text
df4['text'] = df4.apply(lambda row: str_replace(row['text'],row['title']),axis=1)
df4['text'] = df4.apply(lambda row: clean_text(row['text']),axis=1)
df4['text'] = df4.apply(lambda row: remove_urls(row['text']),axis=1)
# Sample text
df4.iloc[3333].text

'ஒற்றை வெளியேற்றப் போட்டி ஓர் ஒற்றைவெளியேற்றப் போட்டியில் singleelimination tournament knockout cup அல்லது sudden death tournament ஒவ்வொரு ஆட்டத்தின் தோற்ற போட்டியாளரும் போட்டியிலிருந்து உடனடியாக விலக்கப்படுகின்ற வகையில் அமைக்கப்படும் போட்டியாகும் இது தோற்ற போட்டியாளர் இனி போட்டியின் எந்தவொரு ஆட்டத்திலும் பங்கு கொள்ள மாட்டார் என்றில்லை சில போட்டிகளில் ஆறுதல் பரிசுக்காகவும் தோற்றவரிடையே முதலிரு நிலைகளை அடுத்த வரிசைகளைத் தீர்மானிக்கவும் தோற்றப் போட்டியாளர்களிடையே ஆட்டங்கள் நடத்தப்படுவதுண்டு இரட்டை வெளியேற்றப் போட்டியில் ஒருவர் இருமுறை தோற்றால் போட்டியிலிருந்து விலக்கப்படுவர்'

In [36]:
# Save it to 
df4.to_csv("data/cleaned_tamil_wiki_text_data.csv")

In [37]:
# Train and test split
train_set,test_set = train_test_split(df4,test_size=0.30,random_state=50)
X_train = train_set.text
X_test = test_set.text
y_train = train_set.parent_category
y_test = test_set.parent_category

In [38]:
# Get a CountVectorizer
tamil_stop_words = get_tamil_stop_words()
vect = CountVectorizer(tokenizer=word_tokenize, stop_words=tamil_stop_words)

In [39]:
# Transform the text into vector
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [40]:
# instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()
# See how the model preforms over various subset of training and test data
scores = cross_val_score(nb, X = X_train_dtm, y = y_train, cv = 5)
scores

array([0.60201401, 0.61254936, 0.62725275, 0.61629956, 0.59858844])

In [41]:
# train/fit the model
%time nb.fit(X_train_dtm, y_train)

CPU times: user 292 ms, sys: 96 ms, total: 388 ms
Wall time: 385 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [42]:
# Show the accuracy of the model for the test data
y_pred_class = nb.predict(X_test_dtm)
metrics.accuracy_score(y_test, y_pred_class)

0.6367179487179487

In [43]:
# Load the external validation dataset
validation_df = pd.read_csv("data/validation.csv")
validation_df['text'] = validation_df.apply(lambda row: clean_text(row['text']),axis=1)
validation_df['text'] = validation_df.apply(lambda row: remove_urls(row['text']),axis=1)
validation_df.head(2)

,Unnamed: 0,title,source,parent_category,text
0,NaN,சுமத்ரான் காண்டாமிருக இனம் முற்றிலும் அழிந்துவ...,https://www.bbc.com/tamil/science-50534560,உயிரியல்,மனிதர்களின் வேட்டைகளினாலும் வாழிட பரப்பு அழிப்...
1,NaN,பிளாஸ்டிக்கில் இருந்து எண்ணெய் எடுக்கும் முயற்...,https://www.bbc.com/tamil/global-50199538,வேதியியல்,தனது ஆய்வகத்தில் உலைக்கலனை கென்னத் போயிப்பெல்ம...


In [44]:
# Convert into vector
X_validation = validation_df.text
y_validation = validation_df.parent_category
X_validation_dtm = vect.transform(X_validation)

In [45]:
# Predict and evaulate the external dataset
y_pred_class2 = nb.predict(X_validation_dtm)
metrics.accuracy_score(y_validation, y_pred_class2)

0.7142857142857143

In [46]:
# Print the external dataset
y_pred_class2

array(['உயிரியல்', 'தொழினுட்பம்', 'வானியல்', 'வானியல்', 'நலம்', 'சட்டம்',
       'சட்டம்', 'உயிரியல்', 'தொழினுட்பம்', 'இயற்பியல்', 'வரலாறு', 'நலம்',
       'பண்பாடு', 'இலக்கியம்'], dtype='<U11')